In [20]:
# Import torch and neural network library:
import torch 
import torch.nn as nn
import torch.nn.functional as F

# import sklearn model_selection, StandardScaler
import sklearn.model_selection
from sklearn.preprocessing import StandardScaler

# import numpy, pandas, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Setup device either gpu or cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Device used: ', device)

Device used:  cpu


In [21]:
# read the dataset framingham.csv and display the first 5 rows.
df = pd.read_csv('framingham.csv')
df.head(5)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [22]:
# display the shape, null values
df.shape

(4240, 16)

In [23]:
df.isna().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [24]:
# Fill null values with either median or mean.

df['BMI'].fillna(df['BMI'].median(), inplace=True) 

meanCol = ['education', 'cigsPerDay', 'BPMeds', 'totChol', 'heartRate', 'glucose']  
for column in meanCol:
    df[column].fillna(df[column].mean(), inplace=True)  

C:\Users\Mandy\AppData\Local\Temp\ipykernel_26460\2739543480.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['BMI'].fillna(df['BMI'].median(), inplace=True)
C:\Users\Mandy\AppData\Local\Temp\ipykernel_26460\2739543480.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [25]:
# get the features as X and the target column as y.
X = df.drop(['TenYearCHD'], axis=1).values
y = df['TenYearCHD'].values

In [26]:
# Split the data into train and test:
train_X, test_X, train_y, test_y = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Scale the features X_train and X_test using StandardScaler
StandardScaler = StandardScaler()
X_train = StandardScaler.fit_transform(train_X)
X_test = StandardScaler.transform(test_X)

# Convert the X_train, X_test, y_train, y_test to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(train_y, dtype=torch.float32).to(device)
y_test = torch.tensor(test_y, dtype=torch.float32).to(device)

In [28]:
# Build your neural network
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(1, 64) 
        self.linear2 = nn.Linear(64, 32)     
        self.linear3 = nn.Linear(32, 16)     
        self.linear4 = nn.Linear(16, 1)       # Adding a second layer to output a single value
        #print(self.linear.weight.shape, self.linear.bias.shape)  # Check the shapes of weight and bias
    def forward(self, x):
        x = F.relu(self.linear1(x))  # Activation function for first layer
        x = F.relu(self.linear2(x))  # Activation function for second layer
        x = F.relu(self.linear3(x))  # Activation function for third layer
        return x

In [29]:
# Instantiate the model, define the loss function and the optimizer
model = MLP().to(device)  # Move the model to the device (GPU or CPU)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
old_loss = None
for i in range(1200):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
        print(f"Loss: {loss**0.5:.4f} at iteration {i+1}")  # Print the square root of loss for better readability

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
# Train the model
for epoch in range(500):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients
    y_pred = model(X_train)  # Forward pass
    loss = loss_fn(y_pred, y_train)  # Compute the loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/500], Loss: {loss.item():.4f}")  # Print the loss every 100 epochs


In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    train_output = model(X_train)
    test_output = model(X_test)
    train_loss = loss(train_output, y_train)
    test_loss = loss (test_output, y_test)
print("Train loss=", train_loss, "test loss=", test_loss)